In [2]:
spark.stop()

In [3]:
import os    
os.environ['SPARK_HOME'] = '/usr/local/lib/python3.9/site-packages/pyspark'

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = (SparkSession
  .builder
  .master("local[*]")
  .appName("np-demo-restapi-pyspark")
  .config("spark.default.parallelism",4)
  .getOrCreate())

# spark.conf.set("spark.default.parallelism",4)

print('Init done')

Init done


21/11/28 07:36:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
file_full_path = '/Users/nguyenpham/Downloads/np-demo-restapi-pyspark/20190207_transactions.json'

df = (spark.read.json(file_full_path))
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [6]:
df.rdd.getNumPartitions()

1

In [15]:
part_df = df.repartition(4)

In [16]:
part_df.rdd.getNumPartitions()

4

In [17]:
from pyspark.sql.functions import explode
explode_df = part_df.select(part_df.id,explode(part_df.products)).withColumnRenamed('col', 'product_id')
explode_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- product_id: long (nullable = true)



In [18]:
df_explode.explain()

== Physical Plan ==
*(2) Project [id#7L, col#18L AS product_id#21L]
+- Generate explode(products#8), [id#7L], false, [col#18L]
   +- Exchange RoundRobinPartitioning(4), REPARTITION_WITH_NUM, [id=#60]
      +- *(1) Filter ((size(products#8, true) > 0) AND isnotnull(products#8))
         +- FileScan json [id#7L,products#8] Batched: false, DataFilters: [(size(products#8, true) > 0), isnotnull(products#8)], Format: JSON, Location: InMemoryFileIndex[file:/Users/nguyenpham/Downloads/np-demo-restapi-pyspark/20190207_transactions...., PartitionFilters: [], PushedFilters: [IsNotNull(products)], ReadSchema: struct<id:bigint,products:array<bigint>>




In [19]:
count_df = explode_df.groupBy('product_id').count()
count_df.printSchema()
count_df.show(5)

root
 |-- product_id: long (nullable = true)
 |-- count: long (nullable = false)

+----------+-----+
|product_id|count|
+----------+-----+
|        29|  112|
|        26|  101|
|        65|  108|
|       191|  113|
|       222|   91|
+----------+-----+
only showing top 5 rows



In [20]:
count_df.rdd.getNumPartitions()

200

In [21]:
count_df.explain()

== Physical Plan ==
*(3) HashAggregate(keys=[product_id#29L], functions=[count(1)])
+- Exchange hashpartitioning(product_id#29L, 200), ENSURE_REQUIREMENTS, [id=#166]
   +- *(2) HashAggregate(keys=[product_id#29L], functions=[partial_count(1)])
      +- *(2) Project [col#26L AS product_id#29L]
         +- Generate explode(products#8), false, [col#26L]
            +- Exchange RoundRobinPartitioning(4), REPARTITION_WITH_NUM, [id=#160]
               +- *(1) Filter ((size(products#8, true) > 0) AND isnotnull(products#8))
                  +- FileScan json [products#8] Batched: false, DataFilters: [(size(products#8, true) > 0), isnotnull(products#8)], Format: JSON, Location: InMemoryFileIndex[file:/Users/nguyenpham/Downloads/np-demo-restapi-pyspark/20190207_transactions...., PartitionFilters: [], PushedFilters: [IsNotNull(products)], ReadSchema: struct<products:array<bigint>>


